In [1]:
import numpy as np
import pandas as pd
import cv2
import os

from tqdm import tqdm

In [2]:
df_img = pd.read_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/03_TextImgMatch/partly/distance.xlsx")

img_folder = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/Test01/02 PyMu_img01"

PDF_folder = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/00_SustainabilityReport_PDF/2022"

# PDF_name = "000999.SZ-华润三九-华润三九 2022年度可持续发展报告-2023-03-29.pdf"

In [3]:
def HSV_calculator(img_path):
    img = cv2.imread(img_path)
    # 将图片从BGR转换为RGB（OpenCV默认使用BGR格式）
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 将RGB图片转换为HSV
    image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)

    mean_hsv = np.mean(image_hsv, axis=(0, 1))
    std_hsv = np.std(image_hsv, axis=(0, 1))

    return mean_hsv, std_hsv, mean_hsv / std_hsv if not(0 in std_hsv) else np.array([0, 0, 0])

In [4]:
def get_bundle_path(directory):
    files = [f for f in os.listdir(directory) if f != '.DS_Store']
    return [os.path.join(directory, file) for file in files if os.path.isfile(os.path.join(directory, file))]

In [5]:
PDF_bundle_path = get_bundle_path(PDF_folder)

for PDF_file_path in PDF_bundle_path:
    PDF_name = os.path.basename(PDF_file_path)
    for img_name in tqdm(df_img.loc[df_img['PDF_name'] == PDF_name, 'file_name']):
        img_path = os.path.join(img_folder, img_name)

        HSV_mean, HSV_std, HSV_vari_coef = HSV_calculator(img_path)

    #     df_img['HSV_mean'] = df_img['HSV_mean'].astype('object')
    #     df_img['HSV_std'] = df_img['HSV_std'].astype('object')
    #     df_img['HSV_vari_coef'] = df_img['HSV_vari_coef'].astype('object')

        df_img.loc[df_img['file_name'] == img_name, 'HSV_mean'] = str(HSV_mean.tolist())
        df_img.loc[df_img['file_name'] == img_name, 'HSV_std'] = str(HSV_std.tolist())
        df_img.loc[df_img['file_name'] == img_name, 'HSV_vari_coef'] = str(HSV_vari_coef.tolist())

100%|█████████████████████████████████████████| 118/118 [00:07<00:00, 15.49it/s]


In [6]:
df_img.to_excel("/Users/improvise/Downloads/HSV_test.xlsx", index=False)